In [113]:
import pandas as pd
import plotly.express as px
from sqlalchemy import create_engine
import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = "notebook"


import locale
locale.setlocale(locale.LC_NUMERIC, 'fr_FR')

engine = create_engine("sqlite:///c:/Users/antoine.herman/Desktop/indicateurs_tdc.sqlite3")

In [2]:
comm_litt = pd.read_csv("C:/Users/antoine.herman/Desktop/communes_litt_mer_cog2021.csv")
comm_litt

,idcom,idcomtxt,iddep
0,97101,ABYMES,971
1,97104,BAILLIF,971
2,97102,ANSE-BERTRAND,971
3,97105,BASSE-TERRE,971
4,97103,BAIE-MAHAULT,971
...,...,...,...
957,85278,SAINT-VINCENT-SUR-JARD,085
958,85288,TALMONT-SAINT-HILAIRE,085
959,85294,LA TRANCHE-SUR-MER,085
960,85297,TRIAIZE,085


In [73]:
national = pd.read_sql("SELECT * FROM indicateurs_national;", con=engine)
national["seuil_frange"] = national["seuil_frange"].astype(str)
national["lbl_frange"] = national["seuil_frange"].apply(lambda x: x + " mètres")

national["p_urba"] = national["surfaces_naf"] / (national["surfaces_naf"] + national["surfaces_urba"])
national["surfaces_naf"] = national["surfaces_naf"] / 10000000
national["surfaces_urba"] = national["surfaces_urba"] / 10000000
national["nb_logt"] = national["nb_logt"] / 1000
national["nb_loc_act"] = national["nb_loc_act"] / 1000

national 

,index,zone,seuil_frange,nb_locaux,nb_logt,estim_logt,estim_loyer,nb_loc_act,estim_bur_com,nb_logt_po,...,nb_commerces,estimation_commerces,nb_campings,nb_hotels,nb_act_autres,surfaces_urba,surfaces_naf,surfaces_habitables,lbl_frange,p_urba
0,0,Littoral,200,1498716.0,920.977,1.968314e+11,1.421946e+08,116.005,13738857541,588055.0,...,56598.0,7309330387,700.0,4475.0,30238.0,49.016652,130.921990,61264705.0,200 mètres,0.727592
1,1,Littoral,1000,4821083.0,3133.015,6.968145e+11,6.310567e+08,320.260,38538129784,1853399.0,...,137456.0,17427617400,2258.0,7949.0,94691.0,176.265247,535.855821,223362345.0,1000 mètres,0.752479
2,2,Littoral,10000,8312551.0,5617.967,1.234510e+12,1.321157e+09,513.305,62275631686,3147746.0,...,197096.0,25200641181,3340.0,9520.0,174961.0,370.045261,2063.987085,418629231.0,10000 mètres,0.847970


In [74]:
color_discrete_map = {"surfaces_naf": "darkgreen", "surfaces_urba": "darkorange"}
new_names = {'surfaces_naf': 'Surface NAF', 'surfaces_urba': 'Surface Urbanisée'}

fig = px.bar(national, 
             y=["surfaces_naf", "surfaces_urba"], 
             x="lbl_frange", 
             color_discrete_map=color_discrete_map,
             text_auto=".2f", 
             orientation="v", 
             height=600, 
             width=700, 
             labels={"value" : "Surface (milliers d'ha)", "lbl_frange": "Bande"},
             title="Répartition des surfaces de foncier concernées en 2021",
             )

for trace, new_name in zip(fig.data, new_names.values()):
    trace.name = new_name
fig.update_layout(legend_title_text="Surface de foncier")
fig.show()

In [75]:

color_discrete_map = {"nb_logt": "indianred", "nb_loc_act": "lightsalmon"}
new_names = {'nb_logt': 'Logements', 'nb_loc_act': "Locaux d'activité"}

fig = px.bar(national, 
             y=["nb_logt", "nb_loc_act"], 
             x="lbl_frange", 
             color_discrete_map=color_discrete_map,
             text_auto=".0f", 
             orientation="v", 
             height=600, 
             width=700, 
             labels={"value" : "Nombre de locaux (milliers)", "lbl_frange": "Bande"},
             title="Ventilation des locaux (hors dépendance)",
             )

for trace, new_name in zip(fig.data, new_names.values()):
    trace.name = new_name
fig.update_layout(legend_title_text="Type de locaux")
fig.show()

In [76]:
df = pd.DataFrame(dict(
    value=[11, 9, 8, 8],
    bande=['200 mètres','1 000 mètres','10 000 mètres',
           'France Entière',]))

fig = px.line_polar(df, 
                    r='value', 
                    theta='bande',
                    line_close=True, 
                    width=550, 
                    height=550, 
                    title="Pourcentage de locaux d'activité selon la bande littorale",
                    )

fig.update_traces(fill='toself')
fig.update_traces(line=dict(color='lightsalmon'))


fig.show()

In [92]:
df = pd.DataFrame(dict(
    surface=[0.3, 1, 4],
    parc=[2.5, 8, 14],
    bande=['200 mètres','1 000 mètres','10 000 mètres',]))

palette = px.colors.sequential.Aggrnyl

color_discrete_map = {"surface": palette[0], "parc": palette[1]}
new_names = {'surface': 'Part de la superficie nationale', 'nb_loc_act': "Part du parc national"}

fig = px.bar(df, 
       x="bande", 
       y=["surface", "parc"], 
       color_discrete_map=color_discrete_map,
       barmode="group", 
       height=600, 
       width=700,
       labels={"value" : "Pourcentage (%)", "bande": "Bande"},
       title="Poids de la superficie et du parc de logements", )
fig.update_traces(texttemplate='%{y:.1f}%', textposition='outside')
for trace, new_name in zip(fig.data, new_names.values()):
    trace.name = new_name
fig.update_layout(legend_title_text="Poids / territoire national")

In [112]:
import plotly.graph_objects as go

national["estim_tot"] = national["estim_logt"] + national["estim_bur_com"]

header=['Bande', 
        'Estimation des logements', 
        "Poids des logements / territoire national", 
        "Estimation des bureaux et commerces", 
        "Estimation globale"]
values = [['200 mètres','1 000 mètres','10 000 mètres',], 
          [str(round(e/1000000000, 2)) + " Md€"  for e in national["estim_logt"].to_list()],
          ["2,5 %", "9 %", "15 %"],
          [str(round(e/1000000000, 2)) + " Md€"  for e in national["estim_bur_com"].to_list()],
          [str(round(e/1000000000, 2)) + " Md€"  for e in national["estim_tot"].to_list()],
          ]

fig = go.Figure(data=[go.Table(
  columnorder = [1,2,3,4,5],
  columnwidth = [200 ,250, 250, 250, 250],
  header =dict(values=header,
    line_color='darkslategray',
    fill_color='royalblue',
    align=["center"]*5,
    font=dict(color='white', size=16),
    height=40
),
  cells=dict(
    values=values,
    line_color='darkslategray',
    fill=dict(color=['paleturquoise', 'white', "white", "white", "darkturquoise"]),
    align=['center',]*5,
    font_size=16,
    height=30)
    )
],
)

fig.update_layout(
    title="Estimation de la valeur vénale des parcs de logement et de locaux d'activité sur le littoral",
    width=1000,
    height=350
)
fig.show()